# 03. AutoML Instance Segmentation Model Testing
This notebook contains sample code to retrieve an instance segmentation model (trained using AutoML for Images) from the AML workspace, deploy that model to an Azure Kubernetes Service cluster (this resource is provisioned if it does not currently exist), then test the real-time endpoint by submitting an HTTP request with image data. Once a response is received, detected instances are annotated and displayed. 

<b><i>Note:</i></b> Only execute cells in this notebook after having run all cells in `01_Setup_AML_Env.ipynb` and `02_Create_AML_Model_Training_Pipeline.ipynb` and once your model training run has completed and successfully added a new model to your registry.

### Import required packages

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset, Model, Run
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE, DEFAULT_GPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

### Connect to AML workspace

In [ ]:
ws = Workspace.from_config()

### Provision an Azure Kubernetes Service inferencing cluster
Models that need to be deployed to real-time endpoints (to handle ad hoc, always-on image scoring) can be deployed to a variety of compute targets - [see this document for more details on different types of AML inference targets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli#choose-a-compute-target). For our demo here, we are provisioning an Azure Kubernetes Service cluster backed by a memory optimized VM type. This inferencing cluster can support large models and high volumes of inferencing traffic. Once the inferencing cluster is provisioned we can deploy our trained instance segmentation model.

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.exceptions import ComputeTargetException

# Choose a name for your cluster
aks_name = "cluster-aks"

# Check to see if the cluster already exists
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print("Found existing compute target. So let's use it")
except ComputeTargetException:
    print('Creating a new compute target...')
    # Provision AKS cluster with GPU machine
    prov_config = AksCompute.provisioning_configuration(vm_size="Standard_DS13-4_v2", 
                                                        location="eastus2")
    # Create the cluster
    aks_target = ComputeTarget.create(workspace=ws, 
                                      name=aks_name, 
                                      provisioning_configuration=prov_config)
    aks_target.wait_for_completion(show_output=True)

### Download and deploy model to local webservice
Trained models registered in your AML workspace can be packaged into a docker container that exposes your model at an API endpoint through a Flask app. This container can be deployed to an authenticated online webservice or run locally. Here we are running this model to our AKS cluster which can be consumed via HTTP requests. See the documents below for tips on troubleshooting deployments by first testing your deployments locally. <b>Note: The step below may take some time to execute (5-15 min).</b>

[Test and Troubleshoot a Local Model Deployment](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-deployment-local)

[Deploy Machine Learning Models to Azure](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice, LocalWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

model_name = 'Street_Segmentation_Model'
deployment_name = 'street-segmentation-model'

model_list = Model.list(ws, model_name)
best_child_run_id = model_list[0].run_id
best_child_run = Run.get(ws, best_child_run_id)
model_list[0].download(exist_ok=True)
model = model_list[0]
environment = Environment.get(ws, 'AutoMLImages_ScoringEnv')
inference_config = InferenceConfig(entry_script='score.py', environment=environment, source_directory='./automl_outputs') # model_path = './automl_outputs/outputs/model.pt'


aks_config = AksWebservice.deploy_configuration(autoscale_enabled=True,                                                    
                                                cpu_cores=1,
                                                memory_gb=50,
                                                enable_app_insights=True)

aks_service = Model.deploy(ws,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           name=deployment_name,
                           overwrite=True)

aks_service.wait_for_deployment(show_output=True)
print()
print("Done. Model is deployed.")
print("\nAKS service status=", aks_service.state)

### Display Test Image

In [ ]:
from IPython.display import Image 
img = Image(filename='./sample_images/SSDB00038.JPG')
img.width = 750
display(img)

### Evaluate and score sample image
Submit an image from the test dataset to the AKS endpoint. The response from the endpoint should contain information about segmented instaces within the image and the code snippet below will annotate your sample image accordingly.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image
import numpy as np
import json
import requests
import cv2

_image = './sample_images/SSDB00038.JPG'
data = open(sample_image, 'rb').read()
# Set the content type
headers = {'Content-Type': 'application/octet-stream', 'Authorization': f'Bearer {aks_service.get_keys()[0]}'}
scoring_uri = aks_service.scoring_uri

# Make the request and display the response
resp = requests.post(scoring_uri, data, headers=headers)

IMAGE_SIZE = (30,20)
plt.figure(figsize=IMAGE_SIZE)
img_np=mpimg.imread(sample_image)
img = Image.fromarray(img_np.astype('uint8'),'RGB')
x, y = img.size

fig,ax = plt.subplots(1, figsize=IMAGE_SIZE)#, figsize=(20,20)
# Display the image
ax.imshow(img_np)

covered_area  = 0.0
total_area  = x * y

covered_area_dict = {}
label_count_dict = {}
ax.set_axis_off()

# draw box and label for each detection 
detections = json.loads(resp.text)
for detect in detections['boxes']:
    label = detect['label']
    box = detect['box']
    polygon = detect['polygon']
    conf_score = detect['score']
    if label not in covered_area_dict.keys():
        covered_area_dict[label] = 0.0
        label_count_dict[label] = 0
    if conf_score > 0.6:
        ymin, xmin, ymax, xmax =  box['topY'],box['topX'], box['bottomY'],box['bottomX']
        topleft_x, topleft_y = x * xmin, y * ymin
        width, height = x * (xmax - xmin), y * (ymax - ymin)
        color = 'dodgerblue'
        if label == 'car':
            color = 'red'
        polygon_np = np.array(polygon[0])
        polygon_np = polygon_np.reshape(-1, 2)
        polygon_np[:, 0] *= x
        polygon_np[:, 1] *= y
        poly = patches.Polygon(polygon_np, True, facecolor=color, alpha=0.25)
        ax.add_patch(poly)
        poly_line = Line2D(polygon_np[:, 0], polygon_np[:, 1], linewidth=2,
                           marker='o', markersize=0, markerfacecolor=color, color=color)
        ax.add_line(poly_line)

        covered_area_dict[label] += cv2.contourArea(np.array(poly.xy).reshape((-1,1,2)).astype(np.int32))
        label_count_dict[label]+=1

plt.show()
display(img)

### Demo Complete! 
This marks the end of the AutoML for Images - Instance Segmentation demo! To continue building upon this sample try labeling your own image dataset using [Azure ML's data labeling tools](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-label-data). Labeled datasets can be exported and passed as inputs to the `PipelineEndpoint` you created earlier in this sample to train new instance segmentation models built around your specific data. These models can then be deployed and consumed as shown above!